In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Allergies"
cohort = "GSE192454"

# Input paths
in_trait_dir = "../../input/GEO/Allergies"
in_cohort_dir = "../../input/GEO/Allergies/GSE192454"

# Output paths
out_data_file = "../../output/preprocess/Allergies/GSE192454.csv"
out_gene_data_file = "../../output/preprocess/Allergies/gene_data/GSE192454.csv"
out_clinical_data_file = "../../output/preprocess/Allergies/clinical_data/GSE192454.csv"
json_path = "../../output/preprocess/Allergies/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Host cellular and immune responses in models of inflammatory skin conditions"
!Series_summary	"Skin colonisation of varied communities of commensal microorganisms, such as Staphylococcus aureus (SA), Staphylococcus epidermidis (SE) and Staphylococcus capitis (SC) form the microbiome; a necessity for healthy skin. The skin changes characteristic of atopic dermatitis, a common inflammatory skin disease, have been shown to provide a favourable niche for SA colonisation. We utilised a reconstructed human epidermal (RHE) model recapitulating the stratified anatomy of the epidermis on which to test host responses to bacterial colonisation. SA proliferation was significantly inhibited in contrast to that seen with SE at both high and low colonisation loads after 24 hours. These data strongly suggest species specific regulation of staphylococcal growth, which is partially mediated by interaction with the epidermis."
!Series_overall_design	"Confluent monol

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on background information, this dataset contains gene expression data from microarray analysis
# of reconstructed human epidermal (RHE) models, so it's likely to contain gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait (Allergies):
# From sample characteristics, we can use 'bacterial_challenge' (index 1) as the trait row
# since we're studying allergies and bacterial colonization is related to skin conditions
# like atopic dermatitis (an allergic condition)
trait_row = 1

# Age: Not available in the dataset
age_row = None

# Gender: Not available in the dataset
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value: str) -> int:
    """
    Convert bacterial challenge information to binary trait values.
    1 = bacterial challenge (potential allergen/inflammatory trigger)
    0 = control (no bacterial challenge)
    """
    if value is None or pd.isna(value) or ":" not in value:
        return None
    
    val = value.split(":", 1)[1].strip().lower()
    
    if "control" in val:
        return 0
    elif any(bacteria in val for bacteria in ["s. aureus", "s. capitis", "s. epidermidis"]):
        return 1
    else:
        return None

def convert_age(value: str) -> float:
    """Placeholder function for age conversion, not used in this dataset."""
    return None

def convert_gender(value: str) -> int:
    """Placeholder function for gender conversion, not used in this dataset."""
    return None

# 3. Save Metadata
# Trait data is available (trait_row = 1), set is_trait_available to True
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    try:
        # The clinical data is likely already loaded in a previous step
        # and might be available in the variable 'clinical_data'
        # If not, load from the appropriate source
        if 'clinical_data' not in locals() or 'clinical_data' not in globals():
            # Assuming clinical_data is available as a variable from previous steps
            # If not, try to load it from the expected format
            try:
                clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
                if os.path.exists(clinical_data_path):
                    clinical_data = pd.read_csv(clinical_data_path)
                else:
                    # If CSV doesn't exist, use the sample characteristics data that was shown in previous output
                    # Create a DataFrame from the sample characteristics dictionary
                    sample_chars = {
                        0: ['tissue_type: reconsituted human epidermis (RHE)'],
                        1: ['bacterial_challenge: Control', 'bacterial_challenge: S. aureus NCTC-8325-4', 
                            'bacterial_challenge: S. capitis ATCC 27840', 
                            'bacterial_challenge: S. aureus ATCC 29213 (NCTC 12973)', 
                            'bacterial_challenge: S. epidermidis ATCC 12228'],
                        2: ['challenge_time_course_hours: 0', 'challenge_time_course_hours: 3', 
                            'challenge_time_course_hours: 24'],
                        3: ['batch_date: 180817', 'batch_date: 80917', 'batch_date: 220917', 'batch_date: 280917'],
                        4: ['array_id: 12342', 'array_id: 12343', 'array_id: 12344', 'array_id: 14525', 
                            'array_id: 14526', 'array_id: 14527', 'array_id: 14576']
                    }
                    # Create a DataFrame from the sample characteristics
                    clinical_data = pd.DataFrame({
                        'key': list(sample_chars.keys()),
                        'value': [sample_chars[k] for k in sample_chars.keys()]
                    })
            except Exception as e:
                print(f"Error loading clinical data: {e}")
                clinical_data = None
        
        if clinical_data is not None:
            # Extract clinical features
            clinical_features = geo_select_clinical_features(
                clinical_df=clinical_data,
                trait=trait,
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age,
                gender_row=gender_row,
                convert_gender=convert_gender
            )
            
            # Preview the extracted features
            print("Preview of clinical features:")
            print(preview_df(clinical_features))
            
            # Save clinical features to CSV
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            clinical_features.to_csv(out_clinical_data_file)
            print(f"Clinical features saved to {out_clinical_data_file}")
        else:
            print("No clinical data available for processing.")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
        print("Continuing with processing without clinical features...")


Preview of clinical features:
{'GSM5747314': [0.0], 'GSM5747315': [1.0], 'GSM5747316': [1.0], 'GSM5747317': [1.0], 'GSM5747318': [0.0], 'GSM5747319': [1.0], 'GSM5747320': [0.0], 'GSM5747321': [1.0], 'GSM5747322': [1.0], 'GSM5747323': [1.0], 'GSM5747324': [1.0], 'GSM5747325': [1.0], 'GSM5747326': [0.0], 'GSM5747327': [1.0], 'GSM5747328': [1.0], 'GSM5747329': [1.0], 'GSM5747330': [1.0], 'GSM5747331': [1.0], 'GSM5747332': [0.0], 'GSM5747333': [1.0], 'GSM5747334': [0.0], 'GSM5747335': [1.0], 'GSM5747336': [1.0], 'GSM5747337': [1.0], 'GSM5747338': [1.0], 'GSM5747339': [0.0], 'GSM5747340': [1.0], 'GSM5747341': [1.0], 'GSM5747342': [1.0], 'GSM5747343': [1.0], 'GSM5747344': [0.0], 'GSM5747345': [1.0]}
Clinical features saved to ../../output/preprocess/Allergies/clinical_data/GSE192454.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '17', '18',
       '19', '20', '21', '22', '23', '24', '25'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers shown are simply numbers (5, 6, 7, 8, etc.)
# These are not standard human gene symbols, which are typically alphabetic 
# characters (like BRCA1, TP53, etc.) or alphanumeric combinations
# These appear to be probe IDs or some other identifiers that would need mapping
# to standard gene symbols for meaningful biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, nan, 'NM_001105533'], 'GB_ACC': [nan, nan, nan, nan, 'NM_001105533'], 'LOCUSLINK_ID': [nan, nan, nan, nan, 79974.0], 'GENE_SYMBOL': [nan, nan, nan, nan, 'CPED1'], 'GENE_NAME': [nan, nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1'], 'UNIGENE_ID': [nan, nan, nan, nan, 'Hs.189652'], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673'], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, 'unmapped', 'chr7:120901888-120901947'], 'CYTOBAND': [nan, nan, na

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the appropriate columns for mapping
# Looking at the gene annotation preview, we see:
# - 'ID' column contains numeric identifiers (1, 2, 3, 4, 5)
# - 'GENE_SYMBOL' column contains gene symbols (e.g., CPED1)
# These match the numeric IDs we saw in the gene expression data

# 2. Create a gene mapping dataframe with the ID and gene symbol columns
mapping_data = get_gene_mapping(gene_annotation, 'ID', 'GENE_SYMBOL')
print(f"Gene mapping dataframe shape: {mapping_data.shape}")
print("Preview of gene mapping data:")
print(preview_df(mapping_data))

# 3. Apply the gene mapping to convert from probe-level to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_data)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("Preview of first 5 genes after mapping:")
print(gene_data.index[:5])


Gene mapping dataframe shape: (51544, 2)
Preview of gene mapping data:
{'ID': ['5', '6', '7', '8', '12'], 'Gene': ['CPED1', 'BCOR', 'CHAC2', 'IFI30', 'GPR146']}
Gene expression data shape after mapping: (19151, 32)
Preview of first 5 genes after mapping:
Index(['A1BG', 'A1BG-AS1', 'A2M', 'A2M-1', 'A2M-AS1'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("Loading the original clinical data...")
# Get the matrix file again to ensure we have the proper data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

print("Extracting clinical features...")
# Use the clinical_data obtained directly from the matrix file
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data using the normalized gene data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from patients with Essential Thrombocythemia (ET), Polycythemia Vera (PV), and Primary Myelofibrosis (PMF)."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Normalizing gene symbols...


Gene data shape after normalization: (16005, 32)


Normalized gene data saved to ../../output/preprocess/Allergies/gene_data/GSE192454.csv
Loading the original clinical data...
Extracting clinical features...
Clinical data preview:
{'GSM5747314': [0.0], 'GSM5747315': [1.0], 'GSM5747316': [1.0], 'GSM5747317': [1.0], 'GSM5747318': [0.0], 'GSM5747319': [1.0], 'GSM5747320': [0.0], 'GSM5747321': [1.0], 'GSM5747322': [1.0], 'GSM5747323': [1.0], 'GSM5747324': [1.0], 'GSM5747325': [1.0], 'GSM5747326': [0.0], 'GSM5747327': [1.0], 'GSM5747328': [1.0], 'GSM5747329': [1.0], 'GSM5747330': [1.0], 'GSM5747331': [1.0], 'GSM5747332': [0.0], 'GSM5747333': [1.0], 'GSM5747334': [0.0], 'GSM5747335': [1.0], 'GSM5747336': [1.0], 'GSM5747337': [1.0], 'GSM5747338': [1.0], 'GSM5747339': [0.0], 'GSM5747340': [1.0], 'GSM5747341': [1.0], 'GSM5747342': [1.0], 'GSM5747343': [1.0], 'GSM5747344': [0.0], 'GSM5747345': [1.0]}
Clinical data saved to ../../output/preprocess/Allergies/clinical_data/GSE192454.csv
Linking clinical and genetic data...
Linked data shape: (32, 

Linked data shape after handling missing values: (32, 16006)
Checking for bias in trait distribution...
For the feature 'Allergies', the least common label is '0.0' with 8 occurrences. This represents 25.00% of the dataset.
The distribution of the feature 'Allergies' in this dataset is fine.

Dataset usability: True


Linked data saved to ../../output/preprocess/Allergies/GSE192454.csv
